In [5]:
import os

import matplotlib

import matplotlib.cm as cm
import matplotlib.animation as animation

import matplotlib.cbook as cbook
import matplotlib.image as image

matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import gdal
import numpy as np
import xarray as xr

import ipywidgets as widgets
import pandas as pd

from IPython.display import clear_output
from IPython.display import display

In [6]:
def get_chunk_size(filename):
    """
    Extract the block size and raster count so that the
    chunks tuple can be formed, a parameter needed to read
    a dataset using xr.open_rasterio() using DASK.
    :param filename: GDAL valid file.
    :return: tuple raster count, x block size, y block size
    """

    # Extract raster count and block size from file
    d = gdal.Open(filename)
    raster_count = d.RasterCount
    # Get internal block size from first band
    b = d.GetRasterBand(1)
    block_size = b.GetBlockSize()
    chunks = (raster_count, block_size[0], block_size[1])

    return chunks

def get_times_from_file_band(fname):
    """
    Extract time info from band metadata
    """
    d = gdal.Open(fname)
    # Get dataset metadata
    dmd = d.GetMetadata()
    bands = d.RasterCount

    # Empty times list
    times = []

    for band in range(bands):
        b = d.GetRasterBand(band+1)
        # Get band metadata
        md = b.GetMetadata()

        # Get fields with date info
        key = 'RANGEBEGINNINGDATE'
        if key in md:
            start_date = md['RANGEBEGINNINGDATE']
        elif key in dmd:
            start_date = dmd['RANGEBEGINNINGDATE']
        else:
            err_msg = f"File {fname} does not have date information"
            raise Exception(err_msg)

        times.append(np.datetime64(start_date))

    return times

In [14]:
# Get monthly ERA5 data
variable = 't2m'
#datadir = '/home/glopez/Projects/ClimateRiskDisclosure/ERA5'
datadir = '../ERA5/US'

fname = os.path.join(datadir, variable, f'{variable}.tif')
print(fname)
times = get_times_from_file_band(fname)

chunks = get_chunk_size(fname)
#data_array = xr.open_rasterio(fname, chunks=chunks)
data_array = xr.open_rasterio(fname)

data_array = data_array.rename(
    {'x': 'longitude',
     'y': 'latitude',
     'band': 'time'})

data_array['time'] = times

../ERA5/US/t2m/t2m.tif


In [13]:
ls ../ERA5/US

t2m/                  u_component_of_wind_10m/  wind_power/
total_precipitation/  v_component_of_wind_10m/


In [ ]:
# Get ERA5 t2m stats from daily data
fname = os.path.join(datadir, variable, f'US_REGIONS_mean_{variable}.tif')
_mean = xr.open_rasterio(fname)

fname = os.path.join(datadir, variable, f'US_REGIONS_std_{variable}.tif')
_std = xr.open_rasterio(fname)

In [ ]:
# Derive the proxy to the Warm Temperature Index
wti = xr.zeros_like(data_array)

In [ ]:
for i, layer in enumerate(data_array):
    wti.data[i] = (data_array.data[i] - _mean.data) / _std.data

In [ ]:
def on_dropdown_dates_change(change):
    """
    Handles change event on dropdown_farms widget
    """
    if change['type'] == 'change' and change['name'] == 'value':
        ax.clear()

        i = np.where(times==dropdown_dates.value)[0][0]
        #img = ax.imshow(wti[i], cmap = 'RdBu_r', vmin=-2.5, vmax=2.5)
        img = wti[i].plot.imshow(ax = ax, vmin=-2.5, vmax=2.5, cmap = 'RdBu_r', add_colorbar=False)
        ax.set_title(f'Monthly {variable} standard anomalies starting on: {times[i].astype(str)}')
        ax.set_aspect('equal')

        plt.show()
        
def on_click(event):
    """
    Event handler
    """
    # Event does not apply for time series plot
    # Check if the click was in a
    if event.inaxes in [bx]:
        return
    
    bx.clear()
    
    # Delete last reference point
    if len(ax.lines) > 0:
        del ax.lines[0]

    # Draw a point as a reference
    ax.plot(event.xdata, event.ydata
            ,marker='o', color='red', markersize=7, alpha=0.7)
    
    _tmp_mean = _mean.sel(x=event.xdata,
                             y=event.ydata,
                             method='nearest')
    
    _data = data_array.sel(longitude=event.xdata,
                           latitude=event.ydata,
                           method='nearest')
    _data.plot(ax=bx, label=f'Monthly ERA5 {variable}', color='Grey')
    
    # Mean
    bx.hlines(_tmp_mean.data[0], data_array.time.data[0], data_array.time.data[-1],
              colors='k', linestyles='-', lw=1.5, label='Mean')   
    
    # 90% and 10% percentile
    _quantile = _data.quantile([0.1,0.9]).data
    bx.hlines(_quantile, data_array.time.data[0], data_array.time.data[-1],
              colors='k', linestyles='--', lw=1.0, label='90% and 10% percentiles')
    
    bx.legend(loc='best', fontsize='small',
                  fancybox=True, framealpha=0.5)
    
    bx.fill_between(data_array.time.data, _quantile[1], _data,
                    where=_data >= _quantile[1],
                    facecolor='red', interpolate=True, alpha=0.5)
    
    bx.fill_between(data_array.time.data, _quantile[0], _data,
                    where=_data <= _quantile[0],
                    facecolor='blue', interpolate=True, alpha=0.5)

    title = bx.get_title()
    bx.set_title(title, fontdict={'horizontalalignment': 'center'})
    
    bx.grid()


In [ ]:
# Plot objects
fig, (ax, bx) = plt.subplots(2, 1)
fig.set_size_inches(10, 8)

# Connect the canvas with the event
cid = fig.canvas.mpl_connect('button_press_event', on_click)

#img = ax.imshow(wti[0], cmap = 'RdBu_r', vmin=-2.5, vmax=2.5)
img = wti[0].plot.imshow(ax = ax, cmap = 'RdBu_r', vmin=-2.5, vmax=2.5, add_colorbar=False)
ax.set_title(f'Monthly {variable} standard anomalies starting on: {times[0].astype(str)}')
ax.set_aspect('equal')
cb = fig.colorbar(img, ax=ax)

bx.grid()

plt.tight_layout()

## Assimila
### ERA 5 2m temperature analysis

In [ ]:
# Dropdown widget with all farms
dropdown_dates = widgets.Dropdown(
    options=times,
    value=times[0],
    description='Date:',
    disabled=False,
)

dropdown_dates.observe(on_dropdown_dates_change)

display(dropdown_dates)
plt.show()